<a href="https://colab.research.google.com/github/zhangxs131/Summary/blob/main/summary_with_hf_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Summary
本文使用huggingface接口，快速实现Summary任务的fine-tune
*   数据，加载datasets中的billsum
*   模型，t5-small

In [1]:
!pip install transformers  datasets

     |████████████████████████████████| 4.0 MB 5.5 MB/s 
     |████████████████████████████████| 325 kB 74.3 MB/s 
     |████████████████████████████████| 6.6 MB 38.3 MB/s 
     |████████████████████████████████| 596 kB 57.4 MB/s 
     |████████████████████████████████| 895 kB 55.9 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 1.1 MB 54.1 MB/s 
     |████████████████████████████████| 212 kB 70.2 MB/s 
     |████████████████████████████████| 136 kB 57.1 MB/s 
     |████████████████████████████████| 127 kB 51.3 MB/s 
     |████████████████████████████████| 271 kB 49.8 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 144 kB 56.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling

In [2]:
from datasets import load_dataset

data=load_dataset('billsum',split='ca_test')
billsum=data.train_test_split(test_size=0.2)
billsum['train'][0]

Using custom data configuration default


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Dataset billsum downloaded and prepared to /root/.cache/huggingface/datasets/billsum/default/3.0.0/d1e95173aed3acb71327864be74ead49b578522e4c7206048b2f2e5351b57959. Subsequent calls will reuse this data.


{'summary': 'Existing law establishes the Public Safety Communications Division within the Office of Emergency Services, under the supervision of a chief, to carry out specific duties relating to state needs and plans for public safety communications systems and equipment.\nThis bill would require the division to require its California 911 Emergency Communications Branch to work with the Department of the California Highway Patrol to continue the work of the Routing on Empirical Data (RED) Project by using the technology and procedures employed in that project to assist in determining whether wireless 911 calls should be routed to a local public safety answering point or a California Highway Patrol call center. The bill would require that the project use historical empirical call data to determine the most efficient routing for wireless 911 calls.\nThe Emergency Telephone Users Surcharge Act generally imposes a surcharge on amounts paid by every person in the state for intrastate telep

In [5]:
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainer,Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq
tokenizer=AutoTokenizer.from_pretrained('t5-small')

prefix='summarize: '
def preprocess_function(example):
  input=[prefix+doc for doc in example['text']]
  model_input=tokenizer(input,max_length=1024,truncation=True)

  with tokenizer.as_target_tokenizer():
    labels=tokenizer(example['summary'],max_length=128,truncation=True)
  model_input['labels']=labels['input_ids']

  return model_input

model=AutoModelForSeq2SeqLM.from_pretrained('t5-small')
tokened_billsum=billsum.map(preprocess_function,batched=True)
data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer,model=model)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
#train

training_args=Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    #fp16=True,
)
trainer=Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokened_billsum['train'],
    eval_dataset=tokened_billsum['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, title, summary. If text, title, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 989
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 62
